In [1]:
import os
import numpy as np
import keras
from keras import layers
from tensorflow import data as tf_data
import tensorflow as tf
#import matplotlib.pyplot as plt
from PIL import Image
from keras.layers import *
from keras.optimizers import *

2024-11-02 11:54:07.488246: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-02 11:54:07.488288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-02 11:54:07.488939: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 11:54:07.495775: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-02 11:54:08.205426: W tensorflow/compiler/tf2

In [2]:
#!python3.9 -m pip install pillow
tf.config.list_physical_devices('GPU') 

2024-11-02 11:54:10.526501: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 11:54:10.550325: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 11:54:10.550634: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
errorCount = 0
DirectoryList = ['Anthurium (Anthurium andraeanum)',
 'Bird of Paradise (Strelitzia reginae)',
 'Boston Fern (Nephrolepis exaltata)',
 'Chinese Money Plant (Pilea peperomioides)',
 'Hyacinth (Hyacinthus orientalis)',
 'Monstera Deliciosa (Monstera deliciosa)',
 'Poinsettia (Euphorbia pulcherrima)',
 'Prayer Plant (Maranta leuconeura)',
 'Tradescantia']
for dirn in DirectoryList:
    for im in os.listdir(f"Data/{dirn}"):
        try:
            image = Image.open(f"Data/{dirn}/{im}")
            new_image = image.resize((128, 128)) 
            new_image.save(f"MoreScaled/{dirn}/{im}") 
        except Exception as e:
            print(e)
            print("Error with image")
            errorCount += 1
    print(f"Total of {errorCount} errors")

cannot write mode RGBA as JPEG
Error with image
Total of 1 errors
Total of 1 errors
cannot write mode RGBA as JPEG
Error with image
Total of 2 errors
cannot write mode RGBA as JPEG
Error with image
cannot write mode RGBA as JPEG
Error with image
Total of 4 errors
Total of 4 errors
cannot write mode RGBA as JPEG
Error with image
Total of 5 errors
cannot write mode RGBA as JPEG
Error with image
Total of 6 errors
Total of 6 errors
Total of 6 errors


In [5]:
for dirnam in os.listdir("Data/"):
    os.mkdir(f"MoreScaled/{dirnam}")

In [3]:
image_size = (256, 256)
batch_size = 4

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "Scaled",
    validation_split=0.2,
    label_mode='categorical',
    subset="both",
    seed=99,
    image_size=image_size,
    batch_size=batch_size,
)

Found 3187 files belonging to 9 classes.
Using 2550 files for training.
Using 637 files for validation.


2024-11-02 11:54:47.792372: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 11:54:47.792602: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 11:54:47.792753: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 11:54:47.953417: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-02 11:54:47.953637: I external/local_xla/xla/stream_executor

In [10]:
model = keras.Sequential()
model.add(Input(shape=(256, 256, 3)))
model.add(Rescaling(1.0/255))
model.add(RandomBrightness(.4, value_range=[0.0, 1.0]))
model.add(RandomFlip(mode="vertical")
model.add(Conv2D(16, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.75))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(9))
model.add(Activation(tf.nn.softmax))

In [11]:
#sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(learning_rate=0.0004)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy', 'CategoricalCrossentropy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 random_brightness (RandomB  (None, 256, 256, 3)       0         
 rightness)                                                      
                                                                 
 conv2d_3 (Conv2D)           (None, 256, 256, 16)      448       
                                                                 
 activation_9 (Activation)   (None, 256, 256, 16)      0         
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 128, 128, 16)      0         
 g2D)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 128, 128, 16)     

In [ ]:
model.fit(train_ds, epochs=20, validation_data=val_ds)

Epoch 1/20


2024-11-02 12:01:34.136710: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_3/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


427/638 [===================>..........] - ETA: 3s - loss: 3.0401 - accuracy: 0.1440 - categorical_crossentropy: 3.0401

2024-11-02 12:01:41.173638: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


531/638 [=======================>......] - ETA: 1s - loss: 2.8722 - accuracy: 0.1540 - categorical_crossentropy: 2.8722

...

638/638 [==============================] - 10s 16ms/step - loss: 1.3568 - accuracy: 0.4369 - categorical_crossentropy: 1.3568 - val_loss: 1.3898 - val_accuracy: 0.5008 - val_categorical_crossentropy: 1.3898
Epoch 17/20
424/638 [==================>...........] - ETA: 3s - loss: 1.3166 - accuracy: 0.4422 - categorical_crossentropy: 1.3166

2024-11-02 12:04:24.421629: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


528/638 [=======================>......] - ETA: 1s - loss: 1.3115 - accuracy: 0.4470 - categorical_crossentropy: 1.3115

2024-11-02 12:04:25.887944: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


638/638 [==============================] - 10s 16ms/step - loss: 1.3102 - accuracy: 0.4431 - categorical_crossentropy: 1.3102 - val_loss: 1.2822 - val_accuracy: 0.5542 - val_categorical_crossentropy: 1.2822
Epoch 18/20
368/638 [================>.............] - ETA: 3s - loss: 1.2389 - accuracy: 0.4946 - categorical_crossentropy: 1.2389

In [22]:
len(list(val_ds))

2024-11-02 11:37:23.984095: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


40

In [59]:
for features, targets in train_ds:
    print(features)

tf.Tensor(
[[[[100. 118.  78.]
   [ 94. 112.  72.]
   [ 77.  93.  57.]
   ...
   [233. 230. 199.]
   [233. 230. 199.]
   [228. 225. 194.]]

  [[ 78.  96.  56.]
   [ 70.  88.  48.]
   [ 99. 115.  78.]
   ...
   [239. 236. 205.]
   [236. 233. 202.]
   [230. 227. 196.]]

  [[ 54.  72.  30.]
   [ 45.  63.  21.]
   [ 78.  96.  56.]
   ...
   [234. 231. 200.]
   [231. 228. 197.]
   [226. 223. 192.]]

  ...

  [[111. 121. 112.]
   [110. 120. 111.]
   [110. 120. 111.]
   ...
   [ 80.  75.  55.]
   [ 83.  78.  58.]
   [ 85.  80.  60.]]

  [[110. 120. 112.]
   [110. 120. 112.]
   [109. 119. 111.]
   ...
   [ 77.  72.  52.]
   [ 79.  74.  54.]
   [ 80.  75.  55.]]

  [[110. 120. 112.]
   [110. 120. 112.]
   [109. 119. 111.]
   ...
   [ 73.  68.  48.]
   [ 74.  69.  49.]
   [ 75.  70.  50.]]]


 [[[255.  18.  83.]
   [255.  18.  81.]
   [251.  23.  82.]
   ...
   [255.  66. 120.]
   [255.  63. 121.]
   [255.  61. 119.]]

  [[255.  26.  86.]
   [253.  25.  84.]
   [250.  28.  85.]
   ...
   [250.  

tf.Tensor(
[[[[  6.   0.   0.]
   [ 31.  22.  25.]
   [ 89.  79.  80.]
   ...
   [115. 113. 126.]
   [ 41.  39.  52.]
   [  0.   0.  11.]]

  [[ 35.  26.  29.]
   [104.  95.  96.]
   [162. 152. 153.]
   ...
   [192. 190. 203.]
   [141. 139. 152.]
   [ 65.  63.  76.]]

  [[104.  94.  95.]
   [154. 144. 145.]
   [180. 170. 171.]
   ...
   [195. 193. 206.]
   [183. 181. 194.]
   [160. 158. 171.]]

  ...

  [[255. 250. 254.]
   [255. 250. 254.]
   [255. 250. 254.]
   ...
   [129.  86.  70.]
   [132.  89.  73.]
   [133.  90.  74.]]

  [[255. 250. 254.]
   [255. 250. 254.]
   [255. 250. 254.]
   ...
   [131.  88.  72.]
   [132.  89.  73.]
   [132.  89.  73.]]

  [[255. 250. 254.]
   [255. 250. 254.]
   [255. 250. 254.]
   ...
   [133.  90.  74.]
   [131.  88.  72.]
   [130.  87.  71.]]]


 [[[226. 232. 232.]
   [227. 233. 233.]
   [228. 232. 233.]
   ...
   [222. 234. 172.]
   [237. 245. 185.]
   [241. 247. 187.]]

  [[226. 232. 232.]
   [227. 233. 233.]
   [228. 232. 233.]
   ...
   [228. 2